<a href="https://colab.research.google.com/github/ebitsdk/ebitsdk/blob/main/ebits_B2C_price_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ebits B2C price calculator

In [72]:
# @title Price calculator code
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Constants
IMPORT_TAX_RATE = 0.05        # 5%
TRANSACTION_FEE_RATE = 0.03   # 3%
SHIPPING_FEE = 0.3            # Conservative shipping rate at 300kr/kg = 0.3kr/gram
VAT_RATE = 1.25               # Danish VAT rate (25%)

# Function to fetch the real-time USD to DKK conversion rate
def fetch_usd_to_dkk_rate():
    api_url = "https://api.exchangerate-api.com/v4/latest/USD"  # Example API (replace with a real one if needed)
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        rates = response.json().get("rates", {})
        return rates.get("DKK", 6.50)  # Fallback to 6.50 if the rate is not found
    except requests.RequestException:
        print("Error fetching exchange rate. Using default rate of 6.50.")
        return 6.50

# Function to calculate tiered price before applying taxes and fees
def calculate_tiered_price(cost_price):
    if cost_price < 10:
        return cost_price * 3.0, "Tier 1: 200% markup"
    elif 10 <= cost_price < 30:
        return cost_price * 2, "Tier 2: 100% markup"
    elif 30 <= cost_price < 150:
        return cost_price * 1.8, "Tier 3: 80% markup"
    else:
        return cost_price * 1.5, "Tier 4: 50% markup"

# Rounding function
def round_up_to_nearest_5_or_9(price):
    rounded_up_5 = ((price // 5) + 1) * 5
    rounded_up_9 = (int(price // 10) * 10 + 9) if price % 10 < 9 else ((int(price // 10) + 1) * 10 + 9)
    if price % 10 == 9 or price == rounded_up_5 - 5:
        return price
    return rounded_up_9 if rounded_up_9 >= price and rounded_up_9 < rounded_up_5 else rounded_up_5

# Function to calculate final values
def calculate_final_price(cost_price, is_usd, batch_size, unit_weight):
    usd_to_dkk_rate = fetch_usd_to_dkk_rate()
    if is_usd:
        cost_price = cost_price * usd_to_dkk_rate
    unit_price = cost_price / batch_size  # Calculate per-unit price

    # add fees to unit price before tiering

    import_tax = unit_price * IMPORT_TAX_RATE
    transaction_fee = unit_price * TRANSACTION_FEE_RATE
    shipping_cost = unit_weight * SHIPPING_FEE if unit_weight > 0 else 0

    unit_price_w_fees = unit_price + import_tax + transaction_fee + shipping_cost
    tiered_price, tier_info = calculate_tiered_price(unit_price_w_fees)


    profit = tiered_price - unit_price_w_fees


    price_with_vat = (unit_price_w_fees + profit ) * VAT_RATE

    final_rounded_price = round_up_to_nearest_5_or_9(price_with_vat)

    final_vat = final_rounded_price * 0.2

    final_profit = final_rounded_price - unit_price_w_fees - final_vat

    COGS = unit_price_w_fees
    revenue = final_rounded_price - final_vat

    profit_margin = ((revenue - COGS) / revenue) * 100

    return {
        "Cost Price per Unit": unit_price,
        "Tiered Price": tiered_price,
        "Tier Info": tier_info,
        "Shipping Cost": shipping_cost,
        "Import Tax": import_tax,
        "Transaction Fee": transaction_fee,
        "Preliminary price with VAT": price_with_vat,
        "Final Profit margin (%)": profit_margin,
        "Final profit per Unit": final_profit,
        "Final VAT:": final_vat,
        "Final Rounded Price per Unit (DKK)": final_rounded_price
    }

# Function to display pie chart
def display_pie_chart(results):
    labels = ["Base Cost", "Import Tax", "Transaction Fee", "VAT", "Profit", "Shipping Cost"]
    base_cost = results["Cost Price per Unit"]
    import_tax = results["Import Tax"]
    transaction_fee = results["Transaction Fee"]
    vat = results["Final VAT:"]
    profit = results["Final profit per Unit"]
    shipping_cost = results["Shipping Cost"]

    sizes = [base_cost, import_tax, transaction_fee, vat, profit, shipping_cost]
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6']

    plt.figure(figsize=(6, 6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors)
    plt.title("Cost Breakdown (Per Unit)")
    plt.axis('equal')  # Equal aspect ratio ensures the pie chart is a circle.
    plt.show()

# UI Elements
usd_input = widgets.FloatText(description="Price in USD:", value=0.0)
dkk_input = widgets.FloatText(description="Price in DKK:", value=0.0)
batch_size_input = widgets.IntText(description="Batch Size:", value=1, min=1)
unit_weight_input = widgets.FloatText(description="Unit Weight(g):", value=0.0)
calculate_button = widgets.Button(description="Calculate", button_style="success")
output = widgets.Output()

def on_calculate_button_click(b):
    with output:
        clear_output()  # Clear previous output
        usd_value = usd_input.value
        dkk_value = dkk_input.value
        batch_size = batch_size_input.value
        unit_weight = unit_weight_input.value

        if batch_size <= 0:
            print("Error: Batch size must be at least 1.")
            return

        if usd_value > 0 and dkk_value > 0:
            print("Error: Please fill only one field (USD or DKK, not both).")
        elif usd_value > 0:
            results = calculate_final_price(usd_value, is_usd=True, batch_size=batch_size, unit_weight=unit_weight)
        elif dkk_value > 0:
            results = calculate_final_price(dkk_value, is_usd=False, batch_size=batch_size, unit_weight=unit_weight)
        else:
            print("Error: Please enter a value in either USD or DKK.")
            return

        # Display results
        print(f"=== Price Breakdown for Batch Size: {batch_size} ===")
        print(f"Cost Price per Unit: {results['Cost Price per Unit']:.2f} DKK")
        print(f"Price tier: {results['Tier Info']}")

        print("_" * 40)
        print(f"Shipping Cost from China: {results['Shipping Cost']:.2f} DKK")
        print(f"Import Tax: {results['Import Tax']:.2f} DKK")
        print(f"Transaction Fee: {results['Transaction Fee']:.2f} DKK")
        print("_" * 40)

        print(f"Gross Margin: {results['Final Profit margin (%)']:.2f} %")
        print(f"Profit per Unit: {results['Final profit per Unit']:.2f} DKK")
        print(f" VAT: {results['Final VAT:']:.2f} DKK")
        print("_" * 40)
        print(f"Final Price per Unit with VAT: {results['Final Rounded Price per Unit (DKK)']:.2f} DKK")
        print("_" * 40)

        # Display pie chart
        display_pie_chart(results)

# Connect button click event
calculate_button.on_click(on_calculate_button_click)

# Display UI
display(usd_input, dkk_input, batch_size_input, unit_weight_input, calculate_button, output)


FloatText(value=0.0, description='Price in USD:')

FloatText(value=0.0, description='Price in DKK:')

IntText(value=1, description='Batch Size:')

FloatText(value=0.0, description='Unit Weight(g):')

Button(button_style='success', description='Calculate', style=ButtonStyle())

Output()